## Sign up for News API (old Google news API)
* Enter search keyword.
* Adjust from date and end time
* Save results to .csv file

In [1]:
from datetime import datetime, timezone, timedelta
from newsapi import NewsApiClient
import pandas as pd

today = str(datetime)
newsapi = NewsApiClient(api_key='xxx')
search = 'xxx'

from_date = datetime.now(timezone.utc) + timedelta(days=-7)
end_time = datetime.now(timezone.utc) + timedelta(days=0)

all_articles = newsapi.get_everything(q=search,
                                      from_param=from_date,
                                      to=end_time,
                                      language='en',
                                      sort_by='relevancy'
                                      )
# Create Pandas DataFrame
df =pd.DataFrame(all_articles['articles'])
df.to_csv(str(datetime.now(timezone.utc)) + '-' + search + '-'+'news.csv')